# Detecção de faces em vídeos

In [1]:
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import numpy as np
import time

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Leitura do arquivo de vídeo

In [27]:
arquivo_video = '/content/drive/MyDrive/Reconhecimento Facial/videos/video01.mp4'
cap = cv2.VideoCapture(arquivo_video)

In [28]:
conectado, video = cap.read()
print(conectado)

True


In [29]:
video.shape

(1080, 1920, 3)

In [30]:
video_largura = video.shape[1]
video_altura = video.shape[0]
print(video_largura, video_altura)

1920 1080


## Redimensionamento do vídeo (opcional)

In [31]:
largura_maxima = 900

In [32]:
def redimensiona_video(largura, altura, largura_maxima = 600):
  if largura > largura_maxima:
    proporcao = largura / altura
    video_largura = largura_maxima
    video_altura = int(video_largura / proporcao)
  else:
    video_largura = largura
    video_altura = altura
  return video_largura, video_altura

In [33]:
if largura_maxima is not None:
  video_largura, video_altura = redimensiona_video(video_largura, video_altura, largura_maxima)
print(video_largura, video_altura)

900 506


## Configurações do vídeo

In [34]:
arquivo_resultado = 'resultado.avi'

> Codec of the file

*FourCC is a 4-character code of codec used to compress the frames (Example of other codecs: MP4V, MJPG, DIVX, X264...). More options can be seem here: fourcc.org*


In [35]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')

> FPS (frames per second)

If you want to make the video a little slower you can decrease the number of frames per second (to 20 for example)

In [36]:
fps = 24

In [37]:
video_saida = cv2.VideoWriter(arquivo_resultado, fourcc, fps, (video_largura, video_altura))

## Detecção de faces com haarcascade

In [14]:
detector_face_haar = cv2.CascadeClassifier('/content/drive/MyDrive/Reconhecimento Facial/cascades/haarcascade_frontalface_default.xml')

In [15]:
def detecta_face_haar(detector_face, imagem):
  imagem_cinza = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
  deteccoes = detector_face.detectMultiScale(imagem_cinza, scaleFactor = 1.15, minNeighbors = 5, minSize = (50,50))
  for (x, y, w, h) in deteccoes:
    cv2.rectangle(imagem, (x, y), (x + w, y + h), (0, 255, 255), 3)
  return imagem

### Processamento do vídeo

In [16]:
frames_show = 20
frame_atual = 1
max_frames = -1

In [17]:
while cv2.waitKey(1) < 0:
  conectado, frame = cap.read()
  if not conectado:
    break
  if max_frames > -1 and frame_atual > max_frames:
    break
  (H, W) = frame.shape[:2]
  t = time.time()
  if largura_maxima is not None:
    frame = cv2.resize(frame, (video_largura, video_altura))
  frame_processado = detecta_face_haar(detector_face_haar, frame)
  cv2.putText(frame, " frame processado em {:.2f} segundos".format(time.time() - t), (20, video_altura - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (250,250,250), 0, lineType = cv2.LINE_AA)
  video_saida.write(frame_processado)
  if frame_atual <= frames_show:
    cv2_imshow(cv2.resize(frame_processado, (0,0), fx = 0.75, fy = 0.75))
  frame_atual += 1

print('Terminou!')
video_saida.release()
cv2.destroyAllWindows()

Output hidden; open in https://colab.research.google.com to view.

In [18]:
!ffmpeg -y -loglevel panic -i resultado.avi resultado.mp4

In [19]:
def show_video(name_file, width=700, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(name_file, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="Video" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

In [21]:
show_video('resultado.mp4')

Output hidden; open in https://colab.research.google.com to view.

## Detecção de faces com deep learning

In [38]:
network = cv2.dnn.readNetFromCaffe("/content/drive/MyDrive/Reconhecimento Facial/weights/deploy.prototxt.txt", "/content/drive/MyDrive/Reconhecimento Facial/weights/res10_300x300_ssd_iter_140000.caffemodel")

In [39]:
def detecta_face_ssd(net, imagem, show_conf=True, tamanho = 300, conf_min = 0.7):
  (h, w) = imagem.shape[:2]
  blob = cv2.dnn.blobFromImage(cv2.resize(imagem, (tamanho, tamanho)), 1.0, (tamanho, tamanho), (104.0, 117.0, 123.0))
  net.setInput(blob)
  deteccoes = net.forward()

  face = None
  for i in range(0, deteccoes.shape[2]):
    confianca = deteccoes[0, 0, i, 2]
    if confianca > conf_min:
      box = deteccoes[0, 0, i, 3:7] * np.array([w, h, w, h])
      (start_x, start_y, end_x, end_y) = box.astype("int")
  
      cv2.rectangle(imagem, (start_x, start_y), (end_x, end_y), (0, 255, 0), 2)
      if show_conf:
        text_conf = "{:.2f}%".format(confianca * 100)
        cv2.putText(imagem, text_conf, (start_x, start_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 2)

  return imagem

### Processamento do vídeo

In [40]:
while cv2.waitKey(1) < 0:
  conectado, frame = cap.read()
  if not conectado:
    break
  if max_frames > -1 and frame_atual > max_frames:
    break
  (H, W) = frame.shape[:2]
  t = time.time()
  if largura_maxima is not None:
    frame = cv2.resize(frame, (video_largura, video_altura))
  frame_processado = detecta_face_ssd(network, frame, True, 500)
  cv2.putText(frame, " frame processado em {:.2f} segundos".format(time.time() - t), (20, video_altura - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (250,250,250), 0, lineType = cv2.LINE_AA)
  video_saida.write(frame_processado)
  if frame_atual <= frames_show:
    cv2_imshow(cv2.resize(frame_processado, (0,0), fx = 0.75, fy = 0.75))
  frame_atual += 1

print('Terminou!')
video_saida.release()
cv2.destroyAllWindows()

Terminou!


In [41]:
!ffmpeg -y -loglevel panic -i resultado.avi resultado.mp4

In [42]:
show_video('resultado.mp4')

Output hidden; open in https://colab.research.google.com to view.